In [1]:
import ee
import os
import geemap
import time

In [2]:
def get_sentinel_image(
        longitude: float,
        latitude: float,
        tile_size: int,
        init_date: str,
        end_date: str,
):
    center = ee.Geometry.Point([longitude, latitude])

    # Create a buffer around the center point to get a square tile
    tile = center.buffer(tile_size / 2).bounds()

    # Load the Sentinel-2 image collection
    sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(tile) \
        .filterDate(init_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .median()  # Take the median to reduce cloud cover

    # Select the desired bands (e.g., B4, B3, B2 for true color)
    sentinel2_rgb = sentinel2.select(['B4', 'B3', 'B2'])
    return tile, sentinel2_rgb



In [4]:
root_path = "/Users/anamabo/Documents/SegmentWater/"

In [5]:
service_account = "gee-961@genial-airway-429107-d4.iam.gserviceaccount.com"
sa_json_file = os.path.join(root_path, "genial-airway-429107-d4-c315ddc9e712.json")
project_id = "genial-airway-429107-d4"
credentials = ee.ServiceAccountCredentials(service_account, sa_json_file)
ee.Initialize(credentials, project=project_id)

In [8]:
longitude = 5.779670
latitude = 51.502357
tile_size = 2500  # tile size in meters
init_date = "2023-02-01"
end_date = "2023-03-31"

image_tile, sentinel_image = get_sentinel_image(
    longitude=longitude,
    latitude=latitude,
    tile_size=tile_size,
    init_date=init_date,
    end_date=end_date,
)

clipped_image = sentinel_image.clip(image_tile)

In [9]:
# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Display the tile on the map
Map = geemap.Map()
Map.centerObject(image_tile, zoom=15)
Map.addLayer(clipped_image, vis_params, 'Sentinel-2 RGB')
Map.addLayer(image_tile, {}, 'Tile Boundary')

Map

Map(center=[51.50235869326241, 5.779698950904976], controls=(WidgetControl(options=['position', 'transparent_b…